In [12]:
from resnet import resnet32
from cifar import get_cifar10
import utils 
import pencil as pencil_lib

import torchvision.transforms as transforms
import torch
import torch.optim as optim
import torch.nn as nn
from scipy.special import softmax

import copy as cp
import os
import pickle
import numpy as np

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
if True: # Define hyper parameters
    epochs = 320
    batch_size = 128
    lr = 0.06
    momentum = 0.9
    weight_decay = 10e-4
    n_classes = 10
    pencil_epochs = [70, 130, 120]
    pencil_lrs = [0.06, 0.06, 0.06]
    pencil_alpha, pencil_beta, pencil_gamma = 0.1, 0.4, 600
    pencil_KL = True
    pencil_K = 10
    checkpoint = 5
    start_epoch = 0
    seed = 1
    train_ratio = 0.9
    asym_noise = True
    noise_ratio = 0.1
    temp_path = 'PATH_HERE'
    cifar_path = './data'

if True: # Define dataset and transforms
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    np.random.seed(seed)
    trainset, valset = get_cifar10(cifar_path, train_ratio=train_ratio, asym=asym_noise, percent=noise_ratio, train=True, download=False, transform_train=transform_train,
                                               transform_val=transform_val)
    if start_epoch==0: # 
        print('trainset',len(trainset))
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
        valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=4)
######### Model checkpointing commented out
        with open(temp_path+"trainloader"+".pkl", "wb") as file:
            pickle.dump(trainloader,file)
        with open(temp_path+"valloader"+".pkl", "wb") as file:
            pickle.dump(valloader,file)
    else: 
        with open(temp_path+"trainloader"+".pkl", "rb") as file:
            trainloader = pickle.load(file)
        with open(temp_path+"valloader"+".pkl", "rb") as file:
            valloader = pickle.load(file)

if True: # Define Model and optimizer

    # Model
    print("==> creating preact_resnet")
    model = resnet32()
    model = model.to(device)
    print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

if True: # Define dict for storing results
    results = {'train':None,'val':None}
    metrics = ['acc','auprc','auroc','loss']
    for subset in results.keys(): 
        results[subset] = {metric:[] for metric in metrics}

    results['pencil_labels'], results['labels'], results['true_labels'] = [], [], []
    results['pencil_label_drift'], results['pencil_label_correct'] = [], []

    for i in range(len(trainset)): 

        item = trainset.__getitem__(i)
        results['labels'].append(item[1])
        results['true_labels'].append(item[3])

if True: # Define PENCIL
    pencil = pencil_lib.PENCIL(torch.tensor(results['labels']), len(trainset), n_classes, pencil_epochs, pencil_lrs, 
                                               pencil_alpha, pencil_beta, pencil_gamma, save_losses=True, use_KL=pencil_KL, K=pencil_K)

########## Model checkpointing commented out
if start_epoch>0: # Load checkpoint data
    model.load_state_dict(torch.load(temp_path+'epoch'+str(start_epoch)))
    with open(temp_path+"results"+str(start_epoch)+".pkl", "rb") as file:
        results = pickle.load(file)
    with open(temp_path+"pencil"+str(start_epoch)+".pkl", "rb") as file:
        pencil = pickle.load(file)
    pencil.alpha=pencil_alpha
    pencil.beta=pencil_beta

for epoch in range(start_epoch,epochs):
    if True: # Train 
        pencil.set_lr(optimizer, epoch)
        model.train()
        preds, label_list = [], []

        if epoch in pencil_epochs: results['pencil_labels'].append(cp.deepcopy(pencil.y_tilde))

        for batch_idx, (inputs, labels, inds, gtrue_labels) in enumerate(trainloader):
#             print('e')
            inputs, labels = inputs.to(device), labels.to(device, dtype= torch.int64)
            outputs = model(inputs)
            loss = pencil.get_loss(epoch, outputs, labels, inds)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pencil.update_y_tilde(epoch, inds)
            preds.append(outputs.detach())
            label_list.append(labels.cpu())

        label_list = np.vstack(torch.cat(label_list).numpy())
        preds = softmax(np.vstack(torch.cat(preds).cpu().numpy()), axis=1)
        train_results = utils.get_performance_metrics(num_classes=n_classes,preds=preds,label_list=label_list)
        for result in train_results.keys():
            results['train'][result].append(train_results[result])
        label_drift = round(sum(results['labels']==np.argmax(pencil.y_tilde,axis=1))/len(results['labels']),4)
        label_true_corr = round(sum(results['true_labels']==np.argmax(pencil.y_tilde,axis=1))/len(results['true_labels']),4)
        results['pencil_label_drift'].append(label_drift)
        results['pencil_label_correct'].append(label_true_corr)

    if True: # Val
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(valloader):
                inputs, labels = inputs.to(device), labels
                outputs = model(inputs)
                val_preds.append(outputs)
                val_labels.append(labels)
        val_labels = np.vstack(torch.cat(val_labels).numpy())
        val_preds = softmax(np.vstack(torch.cat(val_preds).cpu().numpy()), axis=1)
        val_results = utils.get_performance_metrics(num_classes=n_classes,preds=val_preds,label_list=val_labels)
        for result in val_results.keys():
            results['val'][result].append(val_results[result])

    if True: # Print results 
        print('Epoch', epoch, 'Current LR', round(optimizer.param_groups[0]['lr'],5))
        if epoch>=pencil_epochs[0]:
            print('Percent of estimated labels that have remained the same since end of phase 1',round(sum(np.argmax(results['pencil_labels'][0],axis=1)==np.argmax(pencil.y_tilde,axis=1))/len(results['pencil_labels'][0]),4))
        print('% y_tilde equal to noisy label',results['pencil_label_drift'][-1])
        print('% y_tilde equal to true label',results['pencil_label_correct'][-1])
        print('Train acc %.3f, Train auroc %.3f' % (results['train']['acc'][-1], results['train']['auroc'][-1]))
        print('Val acc %.3f, Val auroc %.3f' % (results['val']['acc'][-1], results['val']['auroc'][-1]))                                                                 

########## Model checkpointing commented out
    if checkpoint!=None and (epoch%checkpoint==0 or epoch+1 in pencil_epochs): #Save model for this epoch
        if epoch%checkpoint==0 and epoch>start_epoch+checkpoint: 
            os.remove(temp_path+'epoch'+str(epoch-checkpoint))
            os.remove(temp_path+"results"+str(epoch-checkpoint)+".pkl")
            os.remove(temp_path+"pencil"+str(epoch-checkpoint)+".pkl")
        torch.save(model.state_dict(), temp_path+'epoch'+str(epoch))
        with open(temp_path+"results"+str(epoch)+".pkl", "wb") as file:
            pickle.dump(results,file)
        with open(temp_path+"pencil"+str(epoch)+".pkl", "wb") as file:
            pickle.dump(pencil,file)

True
<class 'list'>
[3 3 4 ... 0 1 5]
Train: 45000 Val: 5000
trainset 45000
==> creating preact_resnet
Total params: 0.47M
Epoch 0 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.334, Train auroc 0.801
Val acc 0.439, Val auroc 0.873
Epoch 1 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.498, Train auroc 0.885
Val acc 0.536, Val auroc 0.908
Epoch 2 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.591, Train auroc 0.921
Val acc 0.556, Val auroc 0.922
Epoch 3 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.645, Train auroc 0.939
Val acc 0.622, Val auroc 0.939
Epoch 4 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.687, Train auroc 0.951
Val acc 0.703, Val auroc 0.957
Epoch 5 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to

Epoch 51 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.799, Train auroc 0.977
Val acc 0.786, Val auroc 0.979
Epoch 52 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.798, Train auroc 0.977
Val acc 0.789, Val auroc 0.979
Epoch 53 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.796, Train auroc 0.977
Val acc 0.777, Val auroc 0.975
Epoch 54 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.797, Train auroc 0.977
Val acc 0.804, Val auroc 0.982
Epoch 55 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.796, Train auroc 0.977
Val acc 0.793, Val auroc 0.977
Epoch 56 Current LR 0.06
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.799, Train auroc 0.977
Val acc 0.752, Val auroc 0.967
Epoch 57 Current LR 0.06
% y_tilde

C:\Users\Administrator\anaconda3\lib\site-packages\torch\nn\functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch 70 Current LR 0.06
Percent of estimated labels that have remained the same since end of phase 1 1.0
% y_tilde equal to noisy label 1.0
% y_tilde equal to true label 0.95
Train acc 0.836, Train auroc 0.980
Val acc 0.775, Val auroc 0.973


C:\Users\Administrator\anaconda3\lib\site-packages\torch\nn\functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


...

Epoch 317 Current LR 0.0006
Percent of estimated labels that have remained the same since end of phase 1 0.8683
% y_tilde equal to noisy label 0.8683
% y_tilde equal to true label 0.8962
Train acc 0.840, Train auroc 0.973
Val acc 0.861, Val auroc 0.987


C:\Users\Administrator\anaconda3\lib\site-packages\torch\nn\functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch 318 Current LR 0.0006
Percent of estimated labels that have remained the same since end of phase 1 0.8683
% y_tilde equal to noisy label 0.8683
% y_tilde equal to true label 0.8962
Train acc 0.839, Train auroc 0.974
Val acc 0.858, Val auroc 0.987


C:\Users\Administrator\anaconda3\lib\site-packages\torch\nn\functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch 319 Current LR 0.0006
Percent of estimated labels that have remained the same since end of phase 1 0.8683
% y_tilde equal to noisy label 0.8683
% y_tilde equal to true label 0.8962
Train acc 0.839, Train auroc 0.973
Val acc 0.861, Val auroc 0.987


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import os
import numpy as np

def test(model,device):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: %.3f %%' % (
                100.0 * correct / total))

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform_val)
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [16]:
test(model,device)

Accuracy of the network on the 10000 test images: 85.750 %


In [26]:
######### Model checkpointing commented out
print(pencil_epochs)
if checkpoint!=None and (epoch%checkpoint==0 or epoch+1 in pencil_epochs): #Save model for this epoch
    if epoch%checkpoint==0 and epoch>start_epoch+checkpoint: 
        os.remove(temp_path+'epoch'+str(epoch-checkpoint))
        os.remove(temp_path+"results"+str(epoch-checkpoint)+".pkl")
        os.remove(temp_path+"pencil"+str(epoch-checkpoint)+".pkl")
    torch.save(model.state_dict(), temp_path+'epoch'+str(epoch))
    with open(temp_path+"results"+str(epoch)+".pkl", "wb") as file:
        pickle.dump(results,file)
    with open(temp_path+"pencil"+str(epoch)+".pkl", "wb") as file:
        pickle.dump(pencil,file)

[70, 130, 120]


In [1]:
from resnet import resnet32
from cifar import get_cifar10
import utils 
import pencil as pencil_lib

import torchvision.transforms as transforms
import torch
import torch.optim as optim
import torch.nn as nn
from scipy.special import softmax

import copy as cp
import os
import pickle
import numpy as np
if True: # Define hyper parameters
    epochs = 30
    batch_size = 128
    lr = 0.06
    momentum = 0.9
    weight_decay = 10e-4
    n_classes = 10
    pencil_epochs = [70, 130, 120]
    pencil_lrs = [0.06, 0.06, 0.06]
    pencil_alpha, pencil_beta, pencil_gamma = 0.1, 0.4, 600
    pencil_KL = True
    pencil_K = 10
    checkpoint = 5
    start_epoch = 0
    seed = 1
    train_ratio = 0.9
    asym_noise = True
    noise_ratio = 0.2
    temp_path = 'PATH_HERE'
    cifar_path = './data'
if True: # Define dataset and transforms
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    np.random.seed(seed)
    trainset, valset = get_cifar10(cifar_path, train_ratio=train_ratio, asym=asym_noise, percent=noise_ratio, train=True, download=False, transform_train=transform_train,
                                               transform_val=transform_val)

<class 'list'>
[3 3 4 ... 0 1 5]
Train: 45000 Val: 5000
